In [1]:
from decouple import config
import os
import uuid

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.chroma import Chroma

from langchain_ollama import ChatOllama
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from PDFChatBot import PDFChatBot

In [2]:
session_id = str(uuid.uuid4()).replace('-', '_')

In [3]:
if 'OLLAMA_API_BASE_URL' not in os.environ:
    os.environ["OPENAI_API_KEY"] = config('OPENAI_API_KEY')
OLLAMA_API_BASE_URL = os.environ['OLLAMA_API_BASE_URL'] if 'OLLAMA_API_BASE_URL' in os.environ else config('OLLAMA_API_BASE_URL')   
LLM = os.environ['LLM'] if 'LLM' in os.environ else config('LLM')   
EMBEDDING_MODEL = os.environ['EMBEDDING_MODEL'] if 'EMBEDDING_MODEL' in os.environ else config('EMBEDDING_MODEL')  

In [4]:
print(f'Using embedding model: {EMBEDDING_MODEL}')
embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

Using embedding model: sentence-transformers/all-MiniLM-L6-v2


/Users/stolli/miniforge3/envs/llm-app/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/stolli/miniforge3/envs/llm-app/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
print(f'Using LLM: {LLM}')
llm = ChatOllama(
    base_url=OLLAMA_API_BASE_URL, 
    model=LLM
)

Using LLM: deepseek-coder-v2


In [6]:
chat_bot = PDFChatBot('/Users/stolli/IT/Designing Data-Intensive Applications.pdf', embedding_model, llm)

incorrect startxref pointer(1)


Initializing PDF Chatbot ...
--- Loading and vectorizing PDF file ---


parsing for Object Streams


--- Initializing history aware LLM ---


In [7]:
response = chat_bot.get_response('What is the book about? Please summarize it in around 20 sentences. Include a list of the most important topics', session_id=session_id)

--- Generating response ---


{'input': 'What is the book about? Please summarize it in around 20 sentences. Include a list of the most important topics',
 'chat_history': [],
 'context': [Document(metadata={'page': 612, 'source': '/Users/stolli/IT/Designing Data-Intensive Applications.pdf'}, page_content='About the Author\nMartin Kleppmann  is a researcher in distributed systems at the University of Cam‐\nbridge, UK. Previously he was a software engineer and entrepreneur at internet com‐\npanies including LinkedIn and Rapportive, where he worked on large-scale data\ninfrastructure. In the process he learned a few things the hard way, and he hopes this\nbook will save you from repeating the same mistakes.\nMartin is a regular conference speaker, blogger, and open source contributor. He\nbelieves that profound technical ideas should be accessible to everyone, and that\ndeeper understanding will help us develop better software.\nColophon\nThe animal on the cover of Designing Data-Intensive Applications  is an Indian 

In [ ]:
response.answer

In [ ]:
response.context

In [8]:
chat_bot.get_response('What is partitioning?', session_id=session_id)

--- Generating response ---


{'input': 'What is partitioning?',
 'chat_history': [HumanMessage(content='What is the book about? Please summarize it in around 20 sentences. Include a list of the most important topics', additional_kwargs={}, response_metadata={}),
  AIMessage(content=' "Designing Data-Intensive Applications" by Martin Kleppmann explores the fundamental principles and technologies behind modern data-intensive applications, such as databases, stream processing systems, and real-time data infrastructure. The book covers various aspects including traditional relational databases, new architectures like NoSQL and NewSQL, the challenges of scalability and consistency in large-scale systems, and the shift towards more distributed and decentralized approaches. Kleppmann discusses how these technologies enable organizations to handle massive volumes of data efficiently and respond rapidly to changes in information. He also addresses the trade-offs between different architectural choices and provides insights

In [9]:
chat_bot.get_response('Can you repeat the answer as structured list?', session_id=session_id)

--- Generating response ---
